In [1]:
import torch
import numpy as np
import os
from torch import nn
from dataset import CIFAR10_custom
from transforms import CompressedToTensor, ZigZagOrder, ChooseAC, FlattenZigZag
from torchvision.transforms import Compose


torch.set_printoptions(threshold=10_000)
np.set_printoptions(threshold=np.inf)

In [3]:
download_path = os.path.join('..', 'data', 'cifar10')

if not os.path.exists(download_path):
    os.makedirs(download_path)

In [20]:
# transform = Compose([CompressedToTensor(),
#                      ZigZagOrder(),
#                      ChooseAC(5),
#                      FlattenZigZag()])

transform = Compose([CompressedToTensor(),
                     ZigZagOrder(),
                     ChooseAC(5)])



# transform = Compose([CompressedToTensor(),
#                      ZigZagOrder(),
#                      ChooseAC(5)])


cifar = CIFAR10_custom(root=download_path, train=True, transform=transform, target_transform=None, download = False, compression=None)

In [21]:
cifar.to_ycbcr(in_place=True)
cifar.compress(in_place=True)

In [22]:
cifar[0][0].shape

torch.Size([3, 16, 6])

In [23]:
cifar[0][0]

tensor([[[ 37., -13.,  -7.,   3.,  -7.,  -4.],
         [ 42.,   4.,   7.,   0.,  -5.,   1.],
         [ 44.,  -2.,   8.,   0.,  -2.,   4.],
         [ 45.,   8.,   9.,   6.,  -5.,  -2.],
         [ 50.,  12.,   3.,  -2.,  -5.,  -1.],
         [ 55., -11., -23.,  -2.,  16.,   5.],
         [ 72.,   8., -36.,   3.,   2.,  -1.],
         [ 51.,   3.,  -5.,   2.,  -7.,   2.],
         [ 54.,  -6.,  -3.,  -5.,   5.,   8.],
         [ 80.,   7.,  11.,   1.,  -4.,   2.],
         [ 73.,  -6.,  23.,   3.,  -2.,   7.],
         [ 58.,   5.,   2.,   4.,  12.,  13.],
         [ 65.,   4., -12.,   2.,   1.,   5.],
         [ 59.,   1.,   6.,  10.,  -3.,  -1.],
         [ 51.,   4.,  -2.,  -1.,  -1.,  -2.],
         [ 53.,  -3.,   8.,   1.,  -3.,  -5.]],

        [[ 50.,   3.,   3.,   0.,   1.,   1.],
         [ 49.,   0.,   0.,   1.,   1.,   0.],
         [ 50.,  -1.,  -1.,   1.,   1.,   0.],
         [ 50.,  -1.,   0.,   0.,   0.,   1.],
         [ 49.,  -1.,   0.,   0.,   1.,   0.],
         [ 

torch.Size([2, 3, 16, 6])
torch.Size([2, 48, 6])


torch.Size([2, 48, 248])

tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]],

         [[ 9., 10.],
          [11., 12.]],

         [[13., 14.],
          [15., 16.]]],


        [[[17., 18.],
          [19., 20.]],

         [[21., 22.],
          [23., 24.]],

         [[25., 26.],
          [27., 28.]],

         [[29., 30.],
          [31., 32.]]],


        [[[33., 34.],
          [35., 36.]],

         [[37., 38.],
          [39., 40.]],

         [[41., 42.],
          [43., 44.]],

         [[45., 46.],
          [47., 48.]]]])

tensor([[ 1.,  2.,  3.,  4., 17., 18., 19., 20., 33., 34., 35., 36.],
        [ 5.,  6.,  7.,  8., 21., 22., 23., 24., 37., 38., 39., 40.],
        [ 9., 10., 11., 12., 25., 26., 27., 28., 41., 42., 43., 44.],
        [13., 14., 15., 16., 29., 30., 31., 32., 45., 46., 47., 48.]])
torch.Size([4, 12])
